In [1]:
import importlib
import sys
sys.path.insert(0, '/cndd/fangming/CEMBA/snmcseq_dev')

from __init__ import *
from __init__jupyterlab import *
import datetime
from scipy import sparse
import collections
import itertools
import re
import fbpca
import pickle
import scipy.cluster.hierarchy as sch
from sklearn.metrics import euclidean_distances

import snmcseq_utils
importlib.reload(snmcseq_utils)
import CEMBA_run_tsne
import CEMBA_clst_utils
import SCF_utils
importlib.reload(SCF_utils)

import seaborn as sns 
import matplotlib.pyplot as plt

# Configs  

In [3]:
time_stamp = datetime.datetime.now().date()
print(time_stamp)

# config_dir = '/cndd/fangming/CEMBA/data/MOp_all/data_freeze_neurons/' 
# sys.path.insert(0, config_dir)
config_f = 'config_scf_8mods_data_freeze_190917.py' 
exec("from {} import *".format(config_f.split('.')[0]))
output_figures = '/cndd/fangming/CEMBA/data/MOp_all/results/figures/mop_8mods_0915_k30_{{}}_{}.{{}}'.format(time_stamp)

f_joint_clustering = '/cndd/fangming/CEMBA/data/MOp_all/results_final/miniatlas_fig4_scf_clusterings.tsv'
f_cluster_annot = '/cndd/fangming/CEMBA/data/MOp_all/results_final/miniatlas_fig4_scf_annotation_round3.tsv'
f_mod_annot = '/cndd/fangming/CEMBA/data/MOp_all/results_final/miniatlas_datamod_annotation.tsv'

# output_summary_results = '/cndd/fangming/CEMBA/data/MOp_all/results_final/miniatlas_fig4_scf_clusterings.tsv' 
# output_summary_results = os.path.join(outdir, 'intg_summary-{}-2-round-clustering.tsv'.format(name))
# output_summary_results = os.path.join(outdir, 'intg_summary-{}-2-round-clustering.tsv'.format(name))
# print(output_summary_results)

2020-01-09


In [4]:
output_figures

'/cndd/fangming/CEMBA/data/MOp_all/results/figures/mop_8mods_0915_k30_{}_2020-01-09.{}'

In [5]:
# fixed dataset configs
sys.path.insert(0, DATA_DIR)
import __init__datasets
importlib.reload(__init__datasets)
from __init__datasets import *

meta_f = os.path.join(DATA_DIR, '{0}_metadata.tsv')
hvftrs_f = os.path.join(DATA_DIR, '{0}_hvfeatures.{1}')
hvftrs_gene = os.path.join(DATA_DIR, '{0}_hvfeatures.gene')
hvftrs_cell = os.path.join(DATA_DIR, '{0}_hvfeatures.cell')

## Read in data 

In [6]:
log = snmcseq_utils.create_logger()
logging.info('*')

01/09/2020 01:49:41 PM *


In [7]:
metas = collections.OrderedDict()
for mod in mods_selected:
    metas[mod] = pd.read_csv(meta_f.format(mod), sep="\t").reset_index().set_index(settings[mod].cell_col)
    print(mod, metas[mod].shape)

snmcseq_gene (9366, 32)
snatac_gene (54844, 13)


/cndd/fangming/venvs/conda_dobro/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (64) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


smarter_cells (6244, 129)
smarter_nuclei (5911, 146)
10x_cells_v2 (121440, 8)
10x_cells_v3 (69727, 8)
10x_nuclei_v3 (39706, 8)
10x_nuclei_v3_macosko (101647, 19)


## All done, making plots after 

- Plotting packages

In [8]:
# get results summary
mods_ordered = [
    'snatac_gene',
    'snmcseq_gene',
    'smarter_nuclei',
    'smarter_cells',
    '10x_cells_v3',
    '10x_nuclei_v3',
    '10x_nuclei_v3_macosko',
    '10x_cells_v2', 
    ]
assert set(mods_ordered) == set(mods_selected)

f = f_cluster_annot
df_annot = pd.read_csv(f, sep="\t") 

f = f_mod_annot
df_mod_annot = pd.read_csv(f, sep="\t", index_col='Modality') 

f = f_joint_clustering
df_info = pd.read_csv(f, sep="\t") 

# back compatible
df_info = pd.merge(df_info, df_mod_annot[['Name']], left_on='modality', right_index=True)
df_info['modality_name'] = df_info['Name']
df_info = df_info.drop('Name', axis=1)

df_info = pd.merge(df_info, df_annot[['cluster_id_iterative', 'annotation_curated', 'color']], 
                   left_on='joint_cluster_round3', right_on='cluster_id_iterative', )

df_info = df_info.rename(columns={
                'joint_embedding_x': 'tsne_x_joint',
                'joint_embedding_y': 'tsne_y_joint',
                'single_modality_annot': 'annot',
                'single_modality_cluster': 'cluster',
                'annotation_curated': 'joint_cluster_round3_anno',
                'color': 'joint_cluster_round3_color',
            }).drop('cluster_id_iterative', axis=1)
df_info = df_info.set_index('sample')
df_info = df_info.astype({"cluster": str})

print(df_info.shape)
df_info.head()


/cndd/fangming/venvs/conda_dobro/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(408885, 12)


,tsne_x_joint,tsne_y_joint,modality,modality_name,cluster,annot,joint_cluster_round1,joint_cluster_round2,joint_cluster_round3,joint_cluster_round4,joint_cluster_round3_anno,joint_cluster_round3_color
sample,,,,,,,,,,,,
snmcseq_gene_2C_M_0,0.471629,-3.134735,snmcseq_gene,DNA methylation,L4-IT-Rorb_Rorb-Tenm2,L4-IT-Rorb_Rorb-Tenm2,1,1-1,1-1-1,1-1-1-1,L5 IT Rspo1,#3CBC78
snmcseq_gene_2C_M_1011,-0.091199,-2.604921,snmcseq_gene,DNA methylation,L4-IT-Rorb_Rorb-Tenm2,L4-IT-Rorb_Rorb-Tenm2,1,1-1,1-1-1,1-1-1-1,L5 IT Rspo1,#3CBC78
snmcseq_gene_2C_M_1012,-0.037576,-8.106225,snmcseq_gene,DNA methylation,L23-IT-Cux2,L23-IT-Cux2,1,1-1,1-1-1,1-1-1-1,L5 IT Rspo1,#3CBC78
snmcseq_gene_2C_M_1022,0.906631,-0.785923,snmcseq_gene,DNA methylation,L4-IT-Rorb_Rorb-Cpne4,L4-IT-Rorb_Rorb-Cpne4,1,1-1,1-1-1,1-1-1-1,L5 IT Rspo1,#3CBC78
snmcseq_gene_2C_M_1024,1.627603,-1.945875,snmcseq_gene,DNA methylation,L4-IT-Rorb_Rorb-Cpne4,L4-IT-Rorb_Rorb-Cpne4,1,1-1,1-1-1,1-1-1-1,L5 IT Rspo1,#3CBC78


In [9]:
# back compatible
# update settings
settings_new = collections.OrderedDict()
for mod, mod_info in settings.items():
    new_dict = collections.OrderedDict()
    for item_key in mod_info._asdict().keys():
        if item_key == 'name':
            new_dict[item_key] = df_mod_annot.loc[mod, 'Name'] 
        elif item_key == 'color':
            new_dict[item_key] = df_mod_annot.loc[mod, 'Color'] 
        else:
            # unchanged 
            new_dict[item_key] = mod_info._asdict()[item_key]
    
    _items = [val for key, val in new_dict.items()]
    settings_new[mod] = Mod_info(*_items) 
settings = settings_new
# settings

In [11]:
mods_updated = [
    'smarter_cells',
    '10x_cells_v3',
    '10x_cells_v2', 
    'smarter_nuclei',
    '10x_nuclei_v3_macosko',
    '10x_nuclei_v3',
    'snatac_gene',
    'mch',
    'mcg',
]
mods_selected = mods

In [12]:
f_format = '/cndd2/fangming/miniatlas/cluster_level/gene_features_{}_round2.tsv'
centroid_dfs = {}
for mod in mods_selected:
    print(mod)
    if settings[mod].mod_category == 'mc':
        for mod_update in ['mcg', 'mch']:
            f = f_format.format(mod_update)
            centroid_dfs[mod_update] = pd.read_csv(f, sep='\t', index_col=0)
    else:
        f = f_format.format(mod)
        centroid_dfs[mod] = pd.read_csv(f, sep='\t', index_col=0)
    
print(centroid_dfs.keys())
    

snmcseq_gene
snatac_gene
smarter_cells
smarter_nuclei
10x_cells_v2
10x_cells_v3
10x_nuclei_v3
10x_nuclei_v3_macosko
dict_keys(['mcg', 'mch', 'snatac_gene', 'smarter_cells', 'smarter_nuclei', '10x_cells_v2', '10x_cells_v3', '10x_nuclei_v3', '10x_nuclei_v3_macosko'])


In [17]:
np.sum(np.power(10, centroid_dfs['snatac_gene'])-1).head()

1-1    34941.085257
1-2    41201.747687
2-1    59078.235836
2-2    47716.217997
2-3    43136.574581
dtype: float64

In [18]:
np.sum(np.power(10, centroid_dfs['smarter_cells'])-1).head()

1-1     794230.187695
1-2     809351.197483
10-1    877619.386349
2-1     825522.805700
2-2     824513.575142
dtype: float64

In [19]:
np.sum(np.power(10, centroid_dfs['smarter_nuclei'])-1).head()

1-1     315035.399432
1-2     383927.945911
10-1    326362.700332
2-1     324681.294208
2-2     303919.373916
dtype: float64

In [20]:
np.sum(np.power(10, centroid_dfs['10x_cells_v2'])-1).head()

1-1     474279.019549
1-2     567514.035847
10-1    591689.210015
2-1     523467.792720
2-2     583257.270016
dtype: float64